In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from transformers import AutoTokenizer
from molmo_video.preprocessor import MolmoProcessor

tokenizer = AutoTokenizer.from_pretrained("allenai/Molmo-7B-D-0924")


processor = MolmoProcessor(tokenizer=tokenizer, 
                           preprocessor_config_path="/share/users/shehan/workspace_pointing_lmm/MolmoVideo/molmo_video/hf_configs_Molmo-7B-D-0924/preprocessor_config.json")

In [3]:
from molmo_video.preprocessor import (DEFAULT_IMAGE_PATCH_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN, DEFAULT_IM_COL_TOKEN, IMAGE_PROMPT)

In [4]:
SYSTEM_PROMPT = "You are given {num_frames} frames from a video. The frame indices are {selected_frame_idxs}. "
PROMPT_TEMPLATES = [
        "Point to {label}\nPlease say 'This isn't in the video.' if it is not in the video.",
        "Point to all occurrences of \"{label}\"",
        "Point to any {label} in the video",
        "Point to any {label} in the video.",
        "Point: Where are the {label}",
        "Show me where the {label} are",
        "Can you show me where the {label} are?",
        "Show me where the {label} are",
        "Show me where a {label} is",
        "Show me where a {label} is.",
        "If there are any {label} in the video? Show me where they are.",
        "Where are the {label}?",
        "Generate a list of points showing where the {label} are.",
        "Find the \"{label}\".",
        "Find a \"{label}\".",
        "Locate all {label}.",
        "Locate an {label}.",
        "Locate a {label}.",
        "Locate every {label}.",
        "Locate {label}.",
        "Locate the {label}.",
        "Object: {label}\nInstruction: Point to the object.",
        "find {label}",
        "find {label}.",
        "Point to every {label}",
        "find any {label} in the picture",
        "Find the {label}",
        "Find any {label}",
        "Point to a {label}",
        "Point to an {label}",
        "Look for {label} in the video and show me where they are.",
        "Help me find an object in the video by pointing to them.\nObject: {label}.",
        "I am looking for {label}, where can they be found in the video?",
        "Can you see any {label} in the video? Point to them.",
        "Point out each {label} in the video.",
        "Point out every {label} in the video.",
        "Point to the {label} in the video.",
        "Locate each {label} in the video.",
        "Can you point out all {label} in this video?",
        "Please find {label} and show me where they are.",
        "If there are any {label} present, indicate their positions.",
        "If there is a {label} present, indicate its positions.",
        "show me all visible {label}",
    ]

In [5]:
from datasets import load_dataset

# Load the dataset from the JSONL file
dataset = load_dataset("json", 
                       data_files="/share/users/shehan/workspace_pointing_lmm/DataPrep/annotations_davis17_train.jsonl",
                       split="train")

# Print a sample to verify
print(dataset[0])


{'video': 'davis17/00000/frames', 'frame_idxs': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81], 'timestamps': [0.0, 0.03333333333333333, 0.06666666666666667, 0.1, 0.13333333333333333, 0.16666666666666666, 0.2, 0.23333333333333334, 0.26666666666666666, 0.3, 0.3333333333333333, 0.36666666666666664, 0.4, 0.43333333333333335, 0.4666666666666667, 0.5, 0.5333333333333333, 0.5666666666666667, 0.6, 0.6333333333333333, 0.6666666666666666, 0.7, 0.7333333333333333, 0.7666666666666667, 0.8, 0.8333333333333334, 0.8666666666666667, 0.9, 0.9333333333333333, 0.9666666666666667, 1.0, 1.0333333333333334, 1.0666666666666667, 1.1, 1.1333333333333333, 1.1666666666666667, 1.2, 1.2333333333333334, 1.2666666666666666, 1.3, 1.3333333333333333,

In [6]:
dataset

Dataset({
    features: ['video', 'frame_idxs', 'timestamps', 'points', 'caption', 'fps'],
    num_rows: 572
})

In [7]:
# import os
# from PIL import Image

# base_data_dir = '/share/users/shehan/workspace_pointing_lmm/DataPrep'

# def load_images(example):
#     # List to hold the loaded images
#     images = []
#     # Loop over each frame index in the example
#     for idx in example['frame_idxs']:
#         # Construct the full file path for the image.
#         # 'video' contains the directory, e.g. 'davis17/00000/frames'
#         # The image file is assumed to be formatted as "00000.jpg", "00001.jpg", etc.
#         frame_filename = f"{idx:05d}.jpg"
#         frame_path = os.path.join(base_data_dir, example['video'], frame_filename)
        
#         # Open and convert the image to RGB (or keep as-is if desired)
#         try:
#             image = Image.open(frame_path).convert("RGB")
#         except Exception as e:
#             print(f"Error loading image {frame_path}: {e}")
#             image = None
#         images.append(image)
    
#     # Optionally, add the list of images to the example under a new key
#     example["images"] = images
#     return example

# # Use the map function to apply the load_images function to every example in the dataset
# dataset_with_images = dataset.map(load_images)


In [8]:
dataset[0]

{'video': 'davis17/00000/frames',
 'frame_idxs': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81],
 'timestamps': [0.0,
  0.03333333333333333,
  0.06666666666666667,
  0.1,
  0.13333333333333333,
  0.16666666666666666,
  0.2,
  0.23333333333333334,
  0.26666666666666666,
  0.3,
  0.3333333333333333,
  0.36666666666666664,
  0.4,
  0.43333333333333335,
  0.4666666666666667,
  0.5,
  0.5333333333333333,
  0.5666666666666667,
  0.6,
  0.6333333333333333,
  0.6666666666666666,
  0.7,
  0.7333333333333333,
  0.7666666666666667,
  0.8,
  0.8333333333333334,
  0.866666

In [9]:
dataset[0]['points']

{'0': [53.0, 183.30864197530863],
 '1': [52.5, 183.30864197530863],
 '2': [52.25, 182.51851851851853],
 '3': [52.0, 181.72839506172838],
 '4': [51.75, 181.72839506172838],
 '5': [51.5, 181.72839506172838],
 '6': [51.5, 180.93827160493825],
 '7': [51.5, 180.93827160493825],
 '8': [51.75, 180.93827160493825],
 '9': [52.0, 180.93827160493825],
 '10': [52.25, 181.72839506172838],
 '11': [52.5, 181.72839506172838],
 '12': [52.75, 182.51851851851853],
 '13': [53.0, 182.51851851851853],
 '14': [53.25, 183.30864197530863],
 '15': [53.5, 184.0987654320988],
 '16': [53.75, 184.0987654320988],
 '17': [54.25, 184.0987654320988],
 '18': [54.5, 184.88888888888889],
 '19': [55.0, 184.88888888888889],
 '20': [55.5, 185.67901234567904],
 '21': [55.5, 184.0987654320988],
 '22': [55.75, 184.88888888888889],
 '23': [56.0, 185.67901234567904],
 '24': [56.0, 186.46913580246914],
 '25': [56.0, 187.25925925925924],
 '26': [55.75, 188.0493827160494],
 '27': [55.75, 188.8395061728395],
 '28': [55.75, 189.629629

In [10]:
def get_points_in_xml_format(points, caption):
    """
    Convert a list of (t, x, y) points to a string in XML format.
    """
    lines = ["<point"]
    # for t, x, y in points:
    for t, x_y in points.items():
        if x_y is None:
            continue
        x, y = x_y
        # Append each point's attributes on a new, indented line
        lines.append(f' t="{int(t)}" x="{x:.1f}" y="{y:.1f}"')
    # Append the alt attribute and the element's text content on the final line
    lines.append(f' alt="{caption}">{caption}</point>')
    formatted_output = "".join(lines)
    
    return formatted_output

In [11]:
import os
import random
from PIL import Image

base_data_dir = '/share/users/shehan/workspace_pointing_lmm/DataPrep'

num_frames = 4

def random_augmentation(batch):
    """
    Batch-wise random augmentation.
    For each example in the batch:
      - Randomly selects `num_frames` indices (or all if there are fewer).
      - Loads only the images corresponding to the selected frame indices.
      - Subsets the 'frame_idxs', 'timestamps', and 'points' to only those indices.
    Assumes that each example in the batch has keys:
      'video'      : the directory containing the frames,
      'frame_idxs' : a list of frame indices,
      'timestamps' : a list of timestamps,
      'points'     : a list of (t, x, y) values.
    """
    new_batch = {"images": [], "frame_idxs": [], "timestamps": [], "points": [], "question": [], "answer": []}

    # Process each example in the batch
    for i in range(len(batch["video"])):
        video_dir = batch["video"][i]
        frame_idxs = batch["frame_idxs"][i]    # e.g. [0, 1, 2, ...]
        timestamps = batch["timestamps"][i]
        points = batch["points"][i]
        caption = batch["caption"][i]

        # Determine the indices to select
        if len(frame_idxs) <= num_frames:
            selected_indices = list(range(len(frame_idxs)))
        else:
            # Select a sorted list of unique random indices to maintain temporal order
            selected_indices = sorted(random.sample(range(len(frame_idxs)), num_frames))

        # Subset the metadata based on selected indices
        selected_frame_idxs = [frame_idxs[j] for j in selected_indices]
        selected_timestamps = [timestamps[j] for j in selected_indices]
        # selected_points = [points[j] for j in selected_indices]
        # selected_points = [points[j] for j in selected_indices if j in points]
        # print('points', points)
        selected_points = {int(k): v for k, v in points.items() if int(k) in selected_frame_idxs}
        # print('selected_points', selected_points)
        

        # Load images corresponding to the selected frame indices
        images = []
        for j in selected_indices:
            # Construct file path: assuming images are named as "00000.jpg", "00001.jpg", etc.
            frame_filename = f"{frame_idxs[j]:05d}.jpg"
            frame_path = os.path.join(base_data_dir, video_dir, frame_filename)
            try:
                image = Image.open(frame_path).convert("RGB")
            except Exception as e:
                print(f"Error loading image {frame_path}: {e}")
                image = None
            images.append(image)

        # Add the selected data to the new batch
        new_batch["images"].append(images)
        new_batch["frame_idxs"].append(selected_frame_idxs)
        new_batch["timestamps"].append(selected_timestamps)
        new_batch["points"].append(selected_points)
        
        question = SYSTEM_PROMPT.format(num_frames=num_frames, selected_frame_idxs=selected_frame_idxs) + random.choice(PROMPT_TEMPLATES).format(label=caption)
        new_batch["question"].append(question)
        
        answer = get_points_in_xml_format(selected_points, caption)
        new_batch["answer"].append(answer)

    return new_batch

# Now, apply the transform to your dataset:
# dataset = your_huggingface_dataset.with_transform(random_augmentation)
augmented_dataset = dataset.with_transform(random_augmentation)


In [12]:
augmented_dataset

Dataset({
    features: ['video', 'frame_idxs', 'timestamps', 'points', 'caption', 'fps'],
    num_rows: 572
})

In [13]:
augmented_dataset[0:2]

{'images': [[<PIL.Image.Image image mode=RGB size=1024x1024>,
   <PIL.Image.Image image mode=RGB size=1024x1024>,
   <PIL.Image.Image image mode=RGB size=1024x1024>,
   <PIL.Image.Image image mode=RGB size=1024x1024>],
  [<PIL.Image.Image image mode=RGB size=1024x1024>,
   <PIL.Image.Image image mode=RGB size=1024x1024>,
   <PIL.Image.Image image mode=RGB size=1024x1024>,
   <PIL.Image.Image image mode=RGB size=1024x1024>]],
 'frame_idxs': [[9, 40, 57, 61], [3, 24, 26, 43]],
 'timestamps': [[0.3, 1.3333333333333333, 1.9, 2.033333333333333],
  [0.1, 0.8, 0.8666666666666667, 1.4333333333333333]],
 'points': [{9: [52.0, 180.93827160493825],
   40: [55.5, 194.37037037037035],
   57: [54.5, 175.40740740740742],
   61: [53.5, 168.29629629629628]},
  {3: [52.0, 181.72839506172838],
   24: [56.0, 186.46913580246914],
   26: [55.75, 188.0493827160494],
   43: [55.5, 195.1604938271605]}],
 'question': ['You are given 4 frames from a video. The frame indices are [9, 40, 57, 61]. Object: a brown b

In [14]:
import matplotlib.pyplot as plt

In [15]:
import torch

def collate_fn(examples):
    batch_outputs = []
    # Iterate over each example in the batch.
    # for question, answer, images in zip(examples["question"], examples["answer"], examples["image"]):
    
    # print('len(examples["question"])', len(examples["question"]))
    # print('examples["question"]', examples["question"])
    # print('examples["answer"]', examples["answer"])
    
    # print('len(examples)', len(examples))
    # print('examples', examples)
    
    # for n in range(len(examples["question"])):
    for example in examples:
        question = example["question"]
        answer = example["answer"]
        images = example["images"]
        
        print('>>>>>>len(images)', len(images))
        print('>>>>>>images', images[0].size) #dimensions
        
        # if image size is (640, 270), plot
        if images[0].size == (640, 270):
            plt.imshow(images[0])
        
        # question = examples["question"][n]
        # answer = examples["answer"][n]
        # images = examples["images"][n]
        
        conversation = [
            {"role": "user", "content": question},
            {"role": "assistant", "content": answer},
        ]
        prompt = tokenizer.apply_chat_template(
            conversation,
            chat_template=tokenizer.chat_template,
            add_generation_prompt=False,
            tokenize=False,
            return_dict=False
        )
        
        if images:
            example_inputs = processor.process(
                images=images,
                text=prompt,
                for_training=True
            )
        else:
            example_inputs = processor.process(
                text=prompt,
                for_training=True
            )
        
        # Move to cuda and add a batch dimension.
        example_inputs = {k: v.to("cuda").unsqueeze(0) for k, v in example_inputs.items()}
        
        batch_outputs.append(example_inputs)
    
    # Now, collate the list of dictionaries into a single batch dictionary.
    # (Assuming each key in example_inputs is a tensor of the same shape across examples.)
    batch_inputs = {}
    for key in batch_outputs[0]:
        # print('>>> key', key)
        # for bt in batch_outputs:
        #     print('bt[key].shape', bt[key].shape)
        batch_inputs[key] = [ex[key] for ex in batch_outputs] #torch.cat([ex[key] for ex in batch_outputs], dim=0)
    
    # return batch_inputs
    
    '''
    batch_inputs:
    
        input_ids torch.Size([B, 1249])
        images torch.Size([B, 13, 576, 588])
        image_input_idx torch.Size([B, 13, 144])
        image_masks torch.Size([B, 13, 576])
        labels torch.Size([B, 1249])
        
    '''
    
    # batch_inputs has keys: input_ids', 'images', 'image_input_idx', 'image_masks', 'labels'
    
    # print('>> batch_inputs', batch_inputs)
    
    
    # padded_inputs = tokenizer.pad({"input_ids": [ex["input_ids"][0] for ex in batch_inputs],},
    #     padding=True, return_tensors="pt",
    # )
    
    padded_inputs = tokenizer.pad({"input_ids": [ex[0] for ex in batch_inputs['input_ids']],},
        padding=True, return_tensors="pt",
    )
    
    
    # Create the labels and set the special tokens to -100
    labels = padded_inputs["input_ids"].clone()
    labels[labels == tokenizer.pad_token_id] = -100
    
    # Set the special tokens to -100
    labels[labels == processor.special_token_ids[IMAGE_PROMPT]] = -100
    labels[labels == processor.special_token_ids[DEFAULT_IMAGE_PATCH_TOKEN]] = -100
    labels[labels == processor.special_token_ids[DEFAULT_IM_START_TOKEN]] = -100
    labels[labels == processor.special_token_ids[DEFAULT_IM_END_TOKEN]] = -100
    labels[labels == processor.special_token_ids[DEFAULT_IM_COL_TOKEN]] = -100
    
    padded_inputs["labels"] = labels
    
    # padded_inputs["images"] = torch.cat([example["images"] for example in batch_inputs], 0)
    # padded_inputs["image_input_idx"] = torch.cat([example["image_input_idx"] for example in batch_inputs], 0)
    # padded_inputs["image_masks"] = torch.cat([example["image_masks"] for example in batch_inputs], 0)
    
    for ex in batch_inputs['images']:
        print('ex.shape', ex.shape)
    
    padded_inputs["images"] = torch.cat([example for example in batch_inputs["images"]], 0)
    padded_inputs["image_input_idx"] = torch.cat([example for example in batch_inputs["image_input_idx"]], 0)
    padded_inputs["image_masks"] = torch.cat([example for example in batch_inputs["image_masks"]], 0)
    
    padded_inputs = padded_inputs.to(torch.bfloat16).to("cuda")
    
    return padded_inputs


# Model

In [16]:
import os; os.environ['LD_LIBRARY_PATH'] = "/share/softwares/cuda_cudnn/cuda-12.1/lib64:/lib/x86_64-linux-gnu:" + os.environ.get('LD_LIBRARY_PATH', '')


In [17]:

import os

os.environ['CUDA_VISIBLE_DEVICES'] = '4'

In [18]:
import torch

from molmo.model import MolmoForCausalLM


In [19]:
model_id = "allenai/Molmo-7B-D-0924"

In [20]:
USE_QLORA = True

from transformers import BitsAndBytesConfig


# load the model
if USE_QLORA:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )
    model = MolmoForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        quantization_config=bnb_config,
        device_map="auto",
    )
else:
    model = MolmoForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        device_map="auto",
    )

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [21]:
from peft import LoraConfig, get_peft_model

# Define LoRA configuration. By setting target_modules to ["att_proj", "ff_proj"],
# only the transformer (LLM) portion will have LoRA adapters.
lora_config = LoraConfig(
    task_type="CAUSAL_LM",  # for causal language modeling
    r=8,                    # LoRA rank (adjust as needed)
    lora_alpha=8,          # scaling factor
    lora_dropout=0.1,       # dropout probability for LoRA layers
    target_modules=["att_proj", "ff_proj"]
)

# Wrap the model with LoRA (this only affects modules matching the target names)
model = get_peft_model(model, lora_config)

# Optional: print out trainable parameters to verify that only the LLM part is modified.
model.print_trainable_parameters()


trainable params: 11,124,736 || all params: 8,032,150,016 || trainable%: 0.1385


In [22]:
gradient_accumulation_steps = 8

ds_config = {
    "zero_optimization": {
      "stage": 3,
      "offload_param": {
        "device": "cpu"
      },
      "offload_optimizer": {
        "device": "cpu"
      }
    },
    "gradient_accumulation_steps": gradient_accumulation_steps,
    
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    
    "gradient_clipping": "auto",
    
    "bf16": {
      "enabled": True
    }
  }

# from transformers import TrainingArguments
# training_args=TrainingArguments(
#             num_train_epochs=10,
#             remove_unused_columns=False,
#             per_device_train_batch_size=4,
#             gradient_accumulation_steps=gradient_accumulation_steps,
#             warmup_steps=1,
#             learning_rate=2e-5,
#             weight_decay=1e-6,
#             adam_beta2=0.999,

#             save_strategy="no", #TODO: change to 'steps' or 'epoch'
#             # optim="adamw_hf",
#             optim="adamw_torch",
#             push_to_hub=True,
#             save_total_limit=1,
#             bf16=True,
#             output_dir="./lora-molmo-pixmo-video",
            
#             logging_strategy="steps",
#             report_to="wandb",
#             logging_steps=10,
            
#             dataloader_pin_memory=False, 
            
#             deepspeed=ds_config,
#             # distributed_port=29501, #TODO
        # )
        


In [23]:
from trl import SFTConfig

# Configure training arguments
training_args = SFTConfig(
    output_dir="molmo-video",  # Directory to save the model
    num_train_epochs=3,  # Number of training epochs
    per_device_train_batch_size=4,  # Batch size for training
    per_device_eval_batch_size=4,  # Batch size for evaluation
    gradient_accumulation_steps=gradient_accumulation_steps,  # Steps to accumulate gradients
    
    gradient_checkpointing=False,  # Enable gradient checkpointing for memory efficiency
    # Optimizer and scheduler settings
    optim="adamw_torch_fused",  # Optimizer type
    learning_rate=2e-4,  # Learning rate for training
    lr_scheduler_type="constant",  # Type of learning rate scheduler
    # Logging and evaluation
    logging_steps=10,  # Steps interval for logging
    eval_steps=10,  # Steps interval for evaluation
    
    eval_strategy= "no", # "steps",  # Strategy for evaluation
    save_strategy="no", #steps",  # Strategy for saving the model
    
    save_steps=20,  # Steps interval for saving
    metric_for_best_model="eval_loss",  # Metric to evaluate the best model
    greater_is_better=False,  # Whether higher metric values are better
    load_best_model_at_end=True,  # Load the best model after training
    # Mixed precision and gradient settings
    bf16=True,  # Use bfloat16 precision
    tf32=True,  # Use TensorFloat-32 precision
    max_grad_norm=0.3,  # Maximum norm for gradient clipping
    warmup_ratio=0.03,  # Ratio of total steps for warmup
    # Hub and reporting
    push_to_hub=True,  # Whether to push model to Hugging Face Hub
    report_to="wandb",  # Reporting tool for tracking metrics
    # Gradient checkpointing settings
    gradient_checkpointing_kwargs={"use_reentrant": False},  # Options for gradient checkpointing
    # Dataset configuration
    dataset_text_field="",  # Text field in dataset
    dataset_kwargs={"skip_prepare_dataset": True},  # Additional dataset options
    #max_seq_length=1024  # Maximum sequence length for input
    
    deepspeed=ds_config,  # DeepSpeed configuration
)

training_args.remove_unused_columns = False  # Keep unused columns in dataset

In [25]:
# from transformers import Trainer
# trainer = Trainer(
#         model=model,
#         train_dataset=augmented_dataset,
#         data_collator=collate_fn,
#         args=training_args,
#         )

# trainer.train()

from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=augmented_dataset,
    data_collator=collate_fn,
    peft_config=lora_config,
    tokenizer=processor.tokenizer,
)


/tmp/ipykernel_3668347/2984811085.py:13: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


[2025-02-24 21:17:34,196] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [26]:
trainer.train()

Using /home/shehan/.cache/torch_extensions/py310_cu121 as PyTorch extensions root...
Emitting ninja build file /home/shehan/.cache/torch_extensions/py310_cu121/cpu_adam/build.ninja...
Building extension module cpu_adam...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module cpu_adam...


ninja: no work to do.
Time to load cpu_adam op: 2.580343246459961 seconds


RuntimeError: The server socket has failed to listen on any local network address. port: 29500, useIpv6: 0, code: -98, name: EADDRINUSE, message: address already in use

In [ ]:
trainer.save_model("path_to_save_model")
